# HSolo Example

In [ ]:
# Import libraries
from matplotlib import pyplot as plt
import numpy as np
import pyhsolo
import cv2 as cv
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 

In [ ]:
# Load input images
img1 = cv.imread('example_data/vegas1.jpg', cv.IMREAD_UNCHANGED) # Left image
img2 = cv.imread('example_data/vegas2.jpg', cv.IMREAD_UNCHANGED) # Right image

In [ ]:
# Display input images
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 15))

axes[0].imshow(img1[...,::-1], cmap='gray')
axes[1].imshow(img2[...,::-1], cmap='gray')

plt.show()

## Use SIFT to get keypoints

In [ ]:
# Initiate SIFT detector
sift = cv.SIFT_create(contrastThreshold=0.001)

# Find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

# With this high of a match ratio, inlier rate will be around 4%
match_ratio = 0.99

# Instantiate KNN Flann matcher
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 20)
flann = cv.FlannBasedMatcher(index_params, search_params)

# Find keypoint matches
matches = flann.knnMatch(des1,des2,k=2)

# Store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < match_ratio * n.distance:
        good.append(m)
print("Number of Candidate Matches: ", len(good))

## OpenCv Find homography matrix

In [ ]:
# Find homography (OpenCV)
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

homography_matrix, cv_mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 1)

print("OpenCV H:", homography_matrix)
print("OpenCV Num Inliers:", np.sum(cv_mask))

## HSolo Find homography matrix

In [ ]:
# Find homography (HSolo)
N = len(good)
pt1 = np.zeros((N, 2))
pt2 = np.zeros((N, 2))
ori1 = np.zeros((N, 1))
ori2 = np.zeros((N, 1))
q1 = np.zeros((N, 1))
q2 = np.zeros((N, 1))

# Copy over keypoint data 
for ii in range(len(good)):
    
    m = good[ii]
    ori1[ii] = np.radians(kp1[m.queryIdx].angle)
    ori2[ii] = np.radians(kp2[m.trainIdx].angle)

    q1[ii] = kp1[m.queryIdx].size
    q2[ii] = kp2[m.trainIdx].size

    pt1[ii] = kp1[m.queryIdx].pt
    pt2[ii] = kp2[m.trainIdx].pt
    
# Find HSolo homography
err_thresh = 2
H = pyhsolo.findHomography( x1y1=pt1, x2y2=pt2, 
                    scales1=q1, scales2 = q2, 
                    oris1=ori1, oris2=ori2, 
                    threshold=err_thresh, 
                    refine_solution=True,
                    run_inner_ransac_thresh = 30,
                    conf=0.999
                )

homography_matrix_hsolo = H[0]
matches_mask_hsolo = H[1].tolist()

print("HSolo H:", homography_matrix_hsolo)
print("HSolo Num Inliers:", np.sum(matches_mask_hsolo))

## Draw matches OpenCV vs. HSolo

In [ ]:
# Draw OpenCV matches
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = cv_mask.flatten(), # draw only inliers
                   flags = 2)
img_matched = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

In [ ]:
# Draw HSolo matches
draw_params = dict(matchColor = (255,0,0), # draw matches in blue color
                   singlePointColor = None,
                   matchesMask = np.array(matches_mask_hsolo).astype(np.uint8), # draw only inliers
                   flags = 2)
img_matched_hsolo = cv.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)

In [ ]:
# Draw OpenCV & HSolo images for comparison
plt.figure(figsize=(15,15))

plt.imshow(img_matched[...,::-1])
plt.title('OpenCV findHomography Matched inliers - Approximate 4% Inlier Rate')

plt.show()

plt.figure(figsize=(15,15))

plt.imshow(img_matched_hsolo[...,::-1])
plt.title('HSolo Matched inliers - Approximate 4% Inlier Rate')

plt.show()